## **Import Libraries**

In [1]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
import random

In [2]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

random.seed(0)
weight_decay = 0.000005

In [3]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [4]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [5]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [6]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [7]:
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(X_train))
    X_test = normalize_minus1_1(cast_to_floatx(X_test))
    return (X_train, y_train), (X_test, y_test)

In [8]:
def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [9]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [10]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'cifar10': ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [11]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [12]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [13]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [14]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [15]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [16]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [17]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import fashion_mnist, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [18]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print(" Training Class ",single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    #Load the data
    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    #Transformations and Training the classifier
    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    print("START the proposed transformations.................. ")
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print("END the proposed transformations......................... ")
    print("START the training.................. ")

    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=BATCH_SIZE, epochs= N_EPOCHS 
            )
    print("END the training.................. ")
    
    #Estimating the parameters  
    def calc_approx_alpha_sum(observations):
        N = len(observations)
        f = np.mean(observations, axis=0)

        return (N * (len(f) - 1) * (-psi(1))) / (
                N * np.sum(f * np.log(f)) - np.sum(f * np.sum(np.log(observations), axis=0)) - np.sum(f * np.sum(np.log(1-observations), axis=0)) - np.sum([np.log(observations[j][k]/(1-observations[j][k])) for j in range(N)]))
    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x  

    def fixed_point_mul_beta(y,alpha_0,max_iter):
      N = y.shape[0]
      K = y.shape[1]
      y = np.clip(y,10**(-10),0.999)
      log_y = np.log(y)
      log_1_y = np.log(1-y)
      log_y_1_y = np.log(y/(1-y))
      f_y = np.mean(log_y, axis=0)
      f_1_y = np.mean(log_1_y, axis=0)
      f_y_1_y = np.mean(log_y_1_y, axis=0)
      alpha_old = alpha_0
      alpha_new = np.zeros(K)
      for iterr in range(max_iter):
          #calculate alpha
          for i in range(K):
            #aph = psi(np.sum(alpha_old)) + (1/N)*np.sum([np.log(y[j][i]) for j in range(N)]) - (1/N)*np.sum([np.log(1-y[j][i]) for j in range(N)]) - (1/N)*np.sum([np.log(y[j][i]/(1-y[j][i])) for j in range(N)])
            aph = psi(np.sum(alpha_old)) + f_y[i] - f_1_y[i] - (1/N)*np.sum([np.log(1 + np.sum([y[j][l]/(1-y[j][l]) for l in range(K)])) for j in range(N)])
            alpha_new[i] = inv_psi(np.clip(aph,None,150))
            alpha_new[i] = np.clip(alpha_new[i],10**(-10),None)
          if np.sqrt(np.sum((alpha_old - alpha_new) ** 2)) < 10**(-7): 
            break
          #update the parameters
          alpha_old = alpha_new
      return alpha_new

    #Calculating the normality score
    def mul_beta_normality_score(alpha,y):
        N = y.shape[0]
        return np.asarray([np.sum((alpha-1)*np.log(y)[j]) - np.sum((alpha+1)*np.log(1-y)[j]) - np.sum(alpha)*np.log(1+np.sum(y[j]/(1-y[j]))) for j in range(N)]) 
    

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=BATCH_SIZE)
        
        y = observed_dirichlet # Our matrix of observations 
        alpha_0 = 0.1*np.ones(y.shape[1])
        mle_alpha_t = fixed_point_mul_beta(y,alpha_0,max_iter=10)
        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=BATCH_SIZE)
        x_test_p = np.nan_to_num(x_test_p, copy=True, nan=0.5, posinf=None, neginf=None)
        scores += mul_beta_normality_score(mle_alpha_t, x_test_p)

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [19]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    for i in range(n_runs):
        print('Run number:', i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]
        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [20]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 128
N_EPOCHS = 5

In [21]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        #(load_fashion_mnist, 'fashion_mnist', 10),
        (load_cifar10, 'cifar10',10)
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
 Training Class  0
Wide Residual Network-16-8 created.
START the proposed transformations.................. 
END the proposed transformations......................... 
START the training.................. 
Epoch 1/5


204/566 [=========>....................] - ETA: 36:25 - loss: 4.3248 - acc: 0.00 - ETA: 29:32 - loss: 4.3231 - acc: 0.00 - ETA: 25:54 - loss: 4.3220 - acc: 0.01 - ETA: 23:49 - loss: 4.3213 - acc: 0.01 - ETA: 22:52 - loss: 4.3205 - acc: 0.01 - ETA: 22:32 - loss: 4.3201 - acc: 0.01 - ETA: 21:53 - loss: 4.3211 - acc: 0.01 - ETA: 21:44 - loss: 4.3210 - acc: 0.01 - ETA: 21:20 - loss: 4.3217 - acc: 0.01 - ETA: 20:58 - loss: 4.3220 - acc: 0.01 - ETA: 20:41 - loss: 4.3222 - acc: 0.01 - ETA: 20:34 - loss: 4.3215 - acc: 0.01 - ETA: 20:35 - loss: 4.3209 - acc: 0.01 - ETA: 20:21 - loss: 4.3212 - acc: 0.01 - ETA: 20:12 - loss: 4.3213 - acc: 0.01 - ETA: 20:18 - loss: 4.3210 - acc: 0.01 - ETA: 20:08 - loss: 4.3210 - acc: 0.01 - ETA: 19:59 - loss: 4.3206 - acc: 0.01 - ETA: 20:03 - loss: 4.3204 - acc: 0.01 - ETA: 19:55 - loss: 4.3204 - acc: 0.01 - ETA: 19:52 - loss: 4.3202 - acc: 0.01 - ETA: 19:48 - loss: 4.3200 - acc: 0.01 - ETA: 19:42 - loss: 4.3194 - acc: 0.01 - ETA: 19:36 - loss: 4.3192 - acc: 0.01

408/566 [====================>.........] - ETA: 12:41 - loss: 3.6633 - acc: 0.09 - ETA: 12:38 - loss: 3.6562 - acc: 0.09 - ETA: 12:36 - loss: 3.6491 - acc: 0.09 - ETA: 12:34 - loss: 3.6422 - acc: 0.09 - ETA: 12:32 - loss: 3.6359 - acc: 0.09 - ETA: 12:29 - loss: 3.6295 - acc: 0.09 - ETA: 12:27 - loss: 3.6226 - acc: 0.09 - ETA: 12:25 - loss: 3.6156 - acc: 0.09 - ETA: 12:23 - loss: 3.6087 - acc: 0.10 - ETA: 12:20 - loss: 3.6019 - acc: 0.10 - ETA: 12:19 - loss: 3.5952 - acc: 0.10 - ETA: 12:16 - loss: 3.5888 - acc: 0.10 - ETA: 12:14 - loss: 3.5823 - acc: 0.10 - ETA: 12:12 - loss: 3.5753 - acc: 0.10 - ETA: 12:10 - loss: 3.5684 - acc: 0.10 - ETA: 12:08 - loss: 3.5617 - acc: 0.10 - ETA: 12:06 - loss: 3.5549 - acc: 0.10 - ETA: 12:03 - loss: 3.5481 - acc: 0.10 - ETA: 12:01 - loss: 3.5412 - acc: 0.10 - ETA: 11:59 - loss: 3.5347 - acc: 0.11 - ETA: 11:57 - loss: 3.5278 - acc: 0.11 - ETA: 11:54 - loss: 3.5211 - acc: 0.11 - ETA: 11:52 - loss: 3.5142 - acc: 0.11 - ETA: 11:50 - loss: 3.5079 - acc: 0.11

566/566 [==============================] - ETA: 5:30 - loss: 2.6666 - acc: 0.220 - ETA: 5:28 - loss: 2.6636 - acc: 0.220 - ETA: 5:25 - loss: 2.6608 - acc: 0.221 - ETA: 5:23 - loss: 2.6581 - acc: 0.221 - ETA: 5:21 - loss: 2.6550 - acc: 0.222 - ETA: 5:19 - loss: 2.6520 - acc: 0.222 - ETA: 5:17 - loss: 2.6491 - acc: 0.222 - ETA: 5:15 - loss: 2.6460 - acc: 0.223 - ETA: 5:13 - loss: 2.6431 - acc: 0.223 - ETA: 5:10 - loss: 2.6402 - acc: 0.223 - ETA: 5:08 - loss: 2.6372 - acc: 0.224 - ETA: 5:06 - loss: 2.6342 - acc: 0.224 - ETA: 5:04 - loss: 2.6313 - acc: 0.225 - ETA: 5:02 - loss: 2.6284 - acc: 0.225 - ETA: 5:00 - loss: 2.6255 - acc: 0.226 - ETA: 4:58 - loss: 2.6225 - acc: 0.226 - ETA: 4:56 - loss: 2.6198 - acc: 0.226 - ETA: 4:54 - loss: 2.6170 - acc: 0.226 - ETA: 4:51 - loss: 2.6140 - acc: 0.227 - ETA: 4:49 - loss: 2.6114 - acc: 0.227 - ETA: 4:47 - loss: 2.6088 - acc: 0.227 - ETA: 4:45 - loss: 2.6059 - acc: 0.228 - ETA: 4:43 - loss: 2.6031 - acc: 0.228 - ETA: 4:41 - loss: 2.6001 - acc: 0.228

204/566 [=========>....................] - ETA: 20:12 - loss: 1.3189 - acc: 0.37 - ETA: 19:04 - loss: 1.3641 - acc: 0.31 - ETA: 18:47 - loss: 1.3575 - acc: 0.32 - ETA: 18:43 - loss: 1.3212 - acc: 0.33 - ETA: 22:22 - loss: 1.3049 - acc: 0.35 - ETA: 21:49 - loss: 1.3149 - acc: 0.35 - ETA: 21:22 - loss: 1.3273 - acc: 0.35 - ETA: 20:58 - loss: 1.3168 - acc: 0.35 - ETA: 20:38 - loss: 1.3215 - acc: 0.35 - ETA: 20:29 - loss: 1.3330 - acc: 0.35 - ETA: 20:26 - loss: 1.3353 - acc: 0.36 - ETA: 20:19 - loss: 1.3384 - acc: 0.36 - ETA: 20:06 - loss: 1.3358 - acc: 0.36 - ETA: 19:56 - loss: 1.3404 - acc: 0.36 - ETA: 19:47 - loss: 1.3396 - acc: 0.37 - ETA: 19:39 - loss: 1.3398 - acc: 0.37 - ETA: 19:39 - loss: 1.3426 - acc: 0.37 - ETA: 19:31 - loss: 1.3437 - acc: 0.37 - ETA: 19:23 - loss: 1.3426 - acc: 0.37 - ETA: 19:27 - loss: 1.3393 - acc: 0.38 - ETA: 19:20 - loss: 1.3343 - acc: 0.37 - ETA: 19:15 - loss: 1.3248 - acc: 0.38 - ETA: 19:15 - loss: 1.3225 - acc: 0.38 - ETA: 19:10 - loss: 1.3190 - acc: 0.38

...

408/552 [=====================>........] - ETA: 12:07 - loss: 0.7007 - acc: 0.64 - ETA: 12:04 - loss: 0.7007 - acc: 0.64 - ETA: 12:02 - loss: 0.7003 - acc: 0.64 - ETA: 12:00 - loss: 0.6999 - acc: 0.64 - ETA: 11:58 - loss: 0.6993 - acc: 0.64 - ETA: 11:55 - loss: 0.6991 - acc: 0.64 - ETA: 11:53 - loss: 0.6990 - acc: 0.64 - ETA: 11:51 - loss: 0.6993 - acc: 0.64 - ETA: 11:49 - loss: 0.6992 - acc: 0.64 - ETA: 11:47 - loss: 0.6990 - acc: 0.64 - ETA: 11:45 - loss: 0.6992 - acc: 0.65 - ETA: 11:43 - loss: 0.6987 - acc: 0.65 - ETA: 11:41 - loss: 0.6987 - acc: 0.65 - ETA: 11:39 - loss: 0.6981 - acc: 0.65 - ETA: 11:36 - loss: 0.6977 - acc: 0.65 - ETA: 11:34 - loss: 0.6971 - acc: 0.65 - ETA: 11:32 - loss: 0.6970 - acc: 0.65 - ETA: 11:30 - loss: 0.6965 - acc: 0.65 - ETA: 11:27 - loss: 0.6958 - acc: 0.65 - ETA: 11:26 - loss: 0.6957 - acc: 0.65 - ETA: 11:24 - loss: 0.6956 - acc: 0.65 - ETA: 11:21 - loss: 0.6954 - acc: 0.65 - ETA: 11:19 - loss: 0.6947 - acc: 0.65 - ETA: 11:17 - loss: 0.6938 - acc: 0.65

552/552 [==============================] - ETA: 5:00 - loss: 0.6123 - acc: 0.722 - ETA: 4:57 - loss: 0.6117 - acc: 0.723 - ETA: 4:55 - loss: 0.6113 - acc: 0.723 - ETA: 4:53 - loss: 0.6107 - acc: 0.724 - ETA: 4:51 - loss: 0.6098 - acc: 0.724 - ETA: 4:49 - loss: 0.6093 - acc: 0.725 - ETA: 4:47 - loss: 0.6088 - acc: 0.725 - ETA: 4:45 - loss: 0.6081 - acc: 0.725 - ETA: 4:43 - loss: 0.6076 - acc: 0.725 - ETA: 4:41 - loss: 0.6071 - acc: 0.726 - ETA: 4:38 - loss: 0.6065 - acc: 0.726 - ETA: 4:36 - loss: 0.6059 - acc: 0.727 - ETA: 4:34 - loss: 0.6054 - acc: 0.727 - ETA: 4:32 - loss: 0.6046 - acc: 0.727 - ETA: 4:30 - loss: 0.6040 - acc: 0.728 - ETA: 4:28 - loss: 0.6034 - acc: 0.728 - ETA: 4:26 - loss: 0.6028 - acc: 0.728 - ETA: 4:24 - loss: 0.6024 - acc: 0.729 - ETA: 4:22 - loss: 0.6021 - acc: 0.729 - ETA: 4:20 - loss: 0.6017 - acc: 0.729 - ETA: 4:17 - loss: 0.6014 - acc: 0.729 - ETA: 4:15 - loss: 0.6009 - acc: 0.730 - ETA: 4:13 - loss: 0.6005 - acc: 0.730 - ETA: 4:11 - loss: 0.6000 - acc: 0.730

204/552 [==========>...................] - ETA: 18:52 - loss: 0.2376 - acc: 0.94 - ETA: 20:55 - loss: 0.2407 - acc: 0.94 - ETA: 19:23 - loss: 0.2535 - acc: 0.93 - ETA: 18:53 - loss: 0.2449 - acc: 0.94 - ETA: 18:34 - loss: 0.2362 - acc: 0.94 - ETA: 18:59 - loss: 0.2343 - acc: 0.94 - ETA: 18:45 - loss: 0.2399 - acc: 0.94 - ETA: 18:36 - loss: 0.2358 - acc: 0.94 - ETA: 18:40 - loss: 0.2345 - acc: 0.94 - ETA: 18:33 - loss: 0.2393 - acc: 0.94 - ETA: 18:25 - loss: 0.2346 - acc: 0.94 - ETA: 18:26 - loss: 0.2301 - acc: 0.94 - ETA: 18:20 - loss: 0.2306 - acc: 0.94 - ETA: 18:15 - loss: 0.2340 - acc: 0.94 - ETA: 18:10 - loss: 0.2361 - acc: 0.94 - ETA: 18:16 - loss: 0.2369 - acc: 0.94 - ETA: 18:10 - loss: 0.2331 - acc: 0.94 - ETA: 18:06 - loss: 0.2295 - acc: 0.94 - ETA: 18:33 - loss: 0.2273 - acc: 0.94 - ETA: 18:28 - loss: 0.2283 - acc: 0.94 - ETA: 18:24 - loss: 0.2295 - acc: 0.94 - ETA: 18:20 - loss: 0.2304 - acc: 0.94 - ETA: 18:20 - loss: 0.2310 - acc: 0.94 - ETA: 18:16 - loss: 0.2283 - acc: 0.94

408/552 [=====================>........] - ETA: 12:01 - loss: 0.1905 - acc: 0.95 - ETA: 11:59 - loss: 0.1905 - acc: 0.95 - ETA: 11:57 - loss: 0.1903 - acc: 0.95 - ETA: 11:55 - loss: 0.1901 - acc: 0.95 - ETA: 11:53 - loss: 0.1898 - acc: 0.95 - ETA: 11:50 - loss: 0.1895 - acc: 0.95 - ETA: 11:48 - loss: 0.1894 - acc: 0.95 - ETA: 11:46 - loss: 0.1891 - acc: 0.95 - ETA: 11:44 - loss: 0.1888 - acc: 0.96 - ETA: 11:42 - loss: 0.1886 - acc: 0.96 - ETA: 11:40 - loss: 0.1884 - acc: 0.96 - ETA: 11:38 - loss: 0.1887 - acc: 0.96 - ETA: 11:36 - loss: 0.1884 - acc: 0.96 - ETA: 11:33 - loss: 0.1882 - acc: 0.96 - ETA: 11:31 - loss: 0.1879 - acc: 0.96 - ETA: 11:29 - loss: 0.1879 - acc: 0.96 - ETA: 11:29 - loss: 0.1879 - acc: 0.96 - ETA: 11:27 - loss: 0.1876 - acc: 0.96 - ETA: 11:24 - loss: 0.1873 - acc: 0.96 - ETA: 11:22 - loss: 0.1872 - acc: 0.96 - ETA: 11:20 - loss: 0.1869 - acc: 0.96 - ETA: 11:18 - loss: 0.1866 - acc: 0.96 - ETA: 11:16 - loss: 0.1865 - acc: 0.96 - ETA: 11:14 - loss: 0.1864 - acc: 0.96

552/552 [==============================] - ETA: 4:57 - loss: 0.1720 - acc: 0.965 - ETA: 4:55 - loss: 0.1719 - acc: 0.965 - ETA: 4:53 - loss: 0.1718 - acc: 0.966 - ETA: 4:51 - loss: 0.1717 - acc: 0.966 - ETA: 4:49 - loss: 0.1718 - acc: 0.966 - ETA: 4:47 - loss: 0.1716 - acc: 0.966 - ETA: 4:45 - loss: 0.1714 - acc: 0.966 - ETA: 4:42 - loss: 0.1713 - acc: 0.966 - ETA: 4:40 - loss: 0.1713 - acc: 0.966 - ETA: 4:38 - loss: 0.1713 - acc: 0.966 - ETA: 4:36 - loss: 0.1714 - acc: 0.966 - ETA: 4:34 - loss: 0.1714 - acc: 0.966 - ETA: 4:32 - loss: 0.1712 - acc: 0.966 - ETA: 4:30 - loss: 0.1711 - acc: 0.966 - ETA: 4:28 - loss: 0.1710 - acc: 0.966 - ETA: 4:26 - loss: 0.1709 - acc: 0.966 - ETA: 4:24 - loss: 0.1707 - acc: 0.966 - ETA: 4:22 - loss: 0.1706 - acc: 0.966 - ETA: 4:20 - loss: 0.1705 - acc: 0.966 - ETA: 4:18 - loss: 0.1704 - acc: 0.966 - ETA: 4:16 - loss: 0.1703 - acc: 0.966 - ETA: 4:14 - loss: 0.1704 - acc: 0.966 - ETA: 4:11 - loss: 0.1703 - acc: 0.966 - ETA: 4:09 - loss: 0.1703 - acc: 0.966